# GETS Specification Search with GNS DGP


The purpose of this template is to implement and analyze the Elhorst-Vega GETS strategy, starting with the estimation of the GNS specification. Varying values for $\gamma$, $\rho$ and $\lambda$ are considered. This includes spatial Durbin models ($\lambda = 0$), SLX error models ($\rho = 0$), standard SLX regression ($\rho = \lambda = 0$), and for $\gamma = 0$, the standard spatial error ($\rho = 0$), spatial lag ($\lambda = 0$ and standard regression model ($\rho = \lambda = 0$).

The true DGP is GNS, i.e., using dgp_gns. Model estimation depends on the branch in the decision tree. All decisions are based on p-values for the individual t-tests.

This design is any spatial layout.

Note: frequencies are adjusted for actual succesful runs, i.e., out of bounds parameters are removed.

## Modules Used

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.stats import chi2
import time
import spreg
import libpysal
from openpyxl import Workbook
from openpyxl.styles import Font
from openpyxl.formatting.rule import CellIsRule

In [ ]:
print("pd ",pd.__version__)
print("gpd ",gpd.__version__)
print("np ",np.__version__)
print("spreg ",spreg.__version__)
print("libpysal ",libpysal.__version__)

## Test on Common Factor Hypothesis

In [ ]:
def comfac_test(betas,vm):
    """
    Computes the Spatial Common Factor Hypothesis test as shown in Anselin (1988, p. 226-229)

    Parameters
    ----------

    betas       : array
                  coefficient estimates in Spatial Durbin model constant, beta, gamma, lambda
    vm          : array
                  Variance-covariance matrix of the coefficients in Spatial Durbin model
                  Order matches theta' = [constant, beta', gamma', lambda]

    Returns
    -------
    Wld     : float
              Wald statistic
    pvalue  : float
              P value for Wald statistic calculated as a Chi sq. distribution
              with degrees of freedom equal to k-1 (with k as the number of betas)

    """
    kall = betas.shape[0]
    k = int((kall - 2)/2)
    k1 = int(1 + k)
    b = betas[1:k1]
    gam = betas[k1:k1+k]
    lam = betas[-1]
    vv = vm[1:,1:]
    g = lam*b + gam
    G = np.vstack((lam*np.eye(b.shape[0]),np.eye(b.shape[0]),b.T))

    GVGi = np.linalg.inv(np.dot(G.T, np.dot(vv, G)))
    Wld = np.dot(g.T, np.dot(GVGi, g))[0][0]
    df = k
    pvalue = 1-chi2.cdf(Wld, df)
    return Wld, pvalue

## GETS Specification Logic

In [ ]:
def back_spec(y,x, w, wlags= 2, p_value=0.05):
    """
    Backward specification: Starting from the estimation of General Nested Model, 
                            it imposes constraints and test significance of
                            coefficients to suggest the most appropriate model
    
    Arguments:
    ----------
    x: matrix of independent variables
    y: vector of dependent variable
    w: spatial weights matrix 
    wlags: number of spatial lags to use in S2SLS
    p_value= significance threshold
        
    Returns:
    ----------
    result: the suggested DGP according to the backward (general to specific) specification search
    steps: the number of steps required to reach the final suggest
    """

    # methods = ['OLS','SEM','SAR','SDM','SLX','SAREr','SLXEr', 'GNS']

    p=p_value
    
    k = x.shape[1]
    model_gns=spreg.GMM_Error(y,x,w=w,slx_lags=1,add_wy=True,w_lags=wlags)
    pstats = np.array(model_gns.z_stat)[1+k:,1]
    pk = len(pstats)   # number of p-values, last one is p_lam, next to last p_rho, before that p_gam
    
    if pstats.max() < p:   # least significant of three is still significant
        result='GNS'
        paths=1
    elif pstats.min() >= p:  # all non-significant
            result='OLS'
            paths=1
    else:       # at least one non-significant and one sig spatial parameter
                # since max is not sig, but (at least) min is
        cand = pstats.argmax()   # least significant is not sig since max > p
        if cand == (pk-1):    # lambda not significant, but at least one of rho/gamma is
        # go to spatial Durbin - only rho and gam
            model_spd = spreg.GM_Lag(y,x,w=w,slx_lags=1,w_lags=wlags)
            pstats = np.array(model_spd.z_stat)[1+k:,1]
            pk = len(pstats)
            if pstats.max() < p:  # least significant of two is still significant
                # check on spatial common factor
                Wld,pwld = comfac_test(model_spd.betas,model_spd.vm)
                if pwld < p:  # reject common factor hypothesis
                    result='SDM'
                    paths=2
                else:
                    result='SEM'
                    paths=2
            elif pstats.min() >= p:  # none significant, even bother?
                result='OLS'
                paths=2
            else:  # one significant and one non-sign spatial parameter
                cand = pstats.argmax()  # non-significant one
                if cand == (pk - 1):   # rho not sig
                    result='SLX'
                    paths=3
                else: # gamma not sig
                    result='SAR'
                    paths=3
        elif cand == (pk-2):   # rho not significant, but at least one of lambda/gamma is
            # go to SLX-Error
            model_slxerr = spreg.GMM_Error(y,x,w=w,slx_lags=1)
            pstats == np.array(model_slxerr.z_stat)[1+k:,1]
            pk = len(pstats)
            if pstats.max() < p:  # least significant of two is still significant
                result='SLXEr'
                paths=4
            elif pstats.min() >= p:  # none significant, even bother?
                result='OLS'
                paths=4
            else:  # one significant and one non-sign spatial parameter
                cand = pstats.argmax()  # non-significant one
                if cand == (pk - 1):   # lambda not sig
                    result='SLX'
                    paths=5
                else: # gamma not sig
                    result='SEM'
                    paths=5
        else:   # gamma not sig, but at least one of rho/lambda is
            # go to SARSAR
            model_sarsar = spreg.GMM_Error(y,x,w=w,add_wy=True,w_lags=wlags)
            pstats = np.array(model_sarsar.z_stat)[1+k:,1]
            pk = len(pstats)
            if pstats.max() < p:  # least significant of two is still significant
                result='SAREr'
                paths=6
            elif pstats.min() >= p:  # none significant, even bother?
                result='OLS'
                paths=6
            else:  # one significant and one non-sign spatial parameter
                cand = pstats.argmax()  # non-significant one
                if cand == (pk - 1):   # lambda not sig
                    result='SAR'
                    paths=7
                else: # rho not sig
                    result='SEM'
                    paths=7
    return(result,paths)
        


## Specify Data and Weights

### 20x20 square grid - queen contiguity - n=400

In [ ]:
#infileshp = "./data_master/twentwengrid.shp"
#infilew = "./data_master/grid400_q.gal"
#layout = "20x20"

### 40x40 square grid - queen contiguity - n=1600

In [ ]:
#infileshp = "./data_master/fourty40grid.shp"
#infilew = "./data_master/fourty40_q.gal"
#layout = "40x40"

### US Counties - queen contiguity - n=3085

In [ ]:
infileshp = "./data_master/uscounty_nodata.shp"
infilew = "./data_master/uscounty_q.gal"
layout = "US_counties"

### Brazilian Municipios - queen contiguity - n=5568

In [ ]:
#infileshp = "./data_master/Brazil_nodata.shp"
#infilew = "./data_master/Braz_muni_q.gal"
#layout = "BRA_muni"

## Read in Data and Weights

In [ ]:
dfs = gpd.read_file(infileshp)

print(dfs.shape)
print(list(dfs))

w = libpysal.io.open(infilew).read()
w.transform = 'r'
n = w.n
print(n)

## Model Parameters

Parameters set here:

- sample size (n)
- number of replications
- types of weights
- type of error process (SAR or MA)
- beta and gamma parameters
- rho and lambda ranges

Note: the DGP and estimator are hard coded for efficiency (otherwise the if condition would have to be evaluated for each replication)

In [ ]:
# overall random seed
rndseed = 123456789
# number of replications
reps=1000
# error process
errp = 'sar'
#errp = 'ma'
# beta and gamma
b1 = [1,1]
#b1 = [1, 1, 1, 1]
# rho range and lambda range
rho_values = [0, 0.1, 0.3, 0.5, 0.7, 0.9]
lam_values = [0, 0.1, 0.3, 0.5, 0.7, 0.9]
# gamma range
gam_values = [0, -0.5, 0.5]
# result parameter labels
models = ['OLS','SEM','SAR','SDM','SLX','SAREr','SLXEr','GNS']
# Nested Dictionary to store results with the structure Modselect [gam][rho][lam] with model acronym
Modselect = {gam: {rho: {lam: {model: np.zeros(reps) for model in models} for lam in lam_values} 
                   for rho in rho_values} for gam in gam_values}
# Nested dictionary with path selection for Modpaths[gam][rho][lam]
Modpaths = {gam: {rho: {lam: np.zeros(reps,dtype=int) for lam in lam_values} 
                   for rho in rho_values} for gam in gam_values}# 
# Nested dictionary with number of model exceptions for Modexcept[gam][rho][lam]
Modexcept = {gam: {rho: {lam: 0 for lam in lam_values} 
                   for rho in rho_values} for gam in gam_values}
# error-vals not needed, inferred from paths = 0
#error_vals = []

# inverse method - alternative is 'true_inv'
invmethod = 'power_exp'
# p-value
pvalue = 0.01
#pvalue = 0.05
# error distribution
errdist = 'normal'
#errdist = 'lognormal'

# number of explanatory variables
kx = len(b1) - 1

## RHS 

In [ ]:
nk = n*kx
var1 = 12.0/kx
rng=np.random.default_rng(seed=rndseed) # set for X
xx = spreg.dgp.make_x(rng,nk,mu=[0],varu=[var1],method="uniform")
if kx > 1:
    x1 = np.reshape(xx,(n,kx))
else:
    x1 = xx
xb1 = spreg.dgp.make_xb(x1,b1)
wx1 = spreg.dgp.make_wx(x1,w) # default first order


## Print Settings

In [ ]:
print("SETTINGS - GETS Search with GNS DGP - Elhorst/Vega approach")
print("Layout: ",infileshp)
print("Weights: ",infilew)
print("n: ",n)
print("k: ",kx)
print("Error Process: ",errp)
print("Error Distribution: ",errdist)
print("Replications: ",reps)
print("p-value: ",pvalue)
print("Inverse Method: ",invmethod)
print("--------------------------------------")

## Simulation Loop

In [ ]:
t0 = time.time()

if errdist == 'normal':
    vv = 6.0     # var 6 for target R2 of 0.66
elif errdist == 'lognormal':
    vv = 1.1     # var 1.1 for target R2 of 0.66
else:
    print("Error distribution not recognized")     # not used


for gam in gam_values:
    gg=gam
    # create a list with multiple gamma values (all same) when more than one x
    if kx > 1:
        g1 = np.ones(kx)*gg
        g1 = g1.tolist()
    else:
        g1 = gg
    #print("gam ",g1)
    wxg1 = spreg.dgp.make_wxg(wx1,g1) 
    for rho in rho_values:
        rho1=rho
        #print("rho ",rho1)
        for lam in lam_values:
            lam1=lam
            #print("lam ",lam1)
            if not(rho1 + lam1 < 1):
                break
            else:
                print(g1,rho1,lam1)
                rng=np.random.default_rng(seed=rndseed) # reset for simulations
                for i in range (reps):  # no resampling for out of range parameters
                    try:                        
                        u= spreg.dgp.make_error(rng,n,mu=0,varu=vv,method=errdist)  # error distribution as an option

                        # DGP is GNS
                        y1 = spreg.dgp_gns(u,xb1,wxg1,w,rho1,lam1, model= errp)
                        # Run backward specification
                        model_suggested,paths = back_spec(y1, xb1, w)
                        #print("back spec ",model_suggested," paths ",paths)
                        # Append result
                        Modselect[gam][rho][lam][model_suggested][i] = 1
                        Modpaths[gam][rho][lam][i]=paths
                    except:  
                        #error_vals.append([gam,rho,lam])
                        #print("except")
                        Modpaths[gam][rho][lam][i]=0
                        

t1 = time.time()
print("time in minutes: ",(t1-t0)/60.0)

## Search Paths

In [ ]:
for gam in gam_values:
    print("Gam: ",gam)
    for rho in rho_values:
        for lam in lam_values:
            if (rho + lam < 1):
                pfreq = np.zeros(8,dtype=int)  # holder for counts by path
                vp = Modpaths[gam][rho][lam]
                vals,counts = np.unique(vp,return_counts=True)
                pfreq[vals]=counts
                Modexcept[gam][rho][lam]=pfreq[0]
                print(" Rho: ",rho," Lam: ",lam," Path Counts: ",pfreq)
                #print("Modexcept ",Modexcept)

## Graph with Selection Frequencies by Model Type

In [ ]:
lenr = len(rho_values)
lenl = len(lam_values)
for gam in gam_values:
    print("GAMMA: ",gam)
    print("------------")
    for pt in range(len(models)):
        mod = models[pt]
        modsel = np.zeros((lenr,lenl))
        for r in range(lenr):
            rr = lenr - 1 -r
            for c in range(lenl):
                rho = rho_values[r]
                lam = lam_values[c]
                if not(rho+lam < 1):
                    modsel[rr,c]= np.nan
                else:    # divide model selection count by valid models not reps
                    #modsel[rr,c] = np.mean(Modselect[gam][rho][lam][mod])
                    modpicks = Modselect[gam][rho][lam][mod]
                    #print(" modpicks ",modpicks)
                    realreps = reps - Modexcept[gam][rho][lam]
                    #print("realreps ",realreps)
                    if realreps > 0: # check for zero reps
                        modsel[rr,c]= modpicks.sum() / realreps
                    else:
                        modsel[rr,c]= np.nan
        print("Selection Frequency for",mod)
        print(modsel)

### Export to excel file

In [ ]:
results_models={}
# Save dictionary values in lists

for mod in models:
    data = []
    for gam in gam_values:
        for rho in rho_values:
            for lam in lam_values:
                if rho + lam < 1:  
                    modpicks = Modselect[gam][rho][lam][mod]
                    realreps = reps - Modexcept[gam][rho][lam]
                    result= modpicks.sum() / realreps
                    data.append({'gamma': gam, 'rho': rho, 'lambda': lam, 'value': result})

    # Create DataFrame
    df = pd.DataFrame(data)
    # Pivot the DataFrame to get the desired shape
    pivot_df = df.pivot_table(index=['lambda', 'rho'], columns='gamma', values='value', aggfunc='first')
    pivot_df = pivot_df.reset_index()[['rho', 'lambda', 0, -0.5, 0.5]]
    # Save the DataFrame in the dictionary, following a specific order for the excel file
    results_models[mod] = pivot_df.iloc[[0,1,2,3,4,5,6,11,15,18,20,7,8,9,10,12,13,14,16,17,19]]


In [ ]:
#Save in Excel format

with pd.ExcelWriter(f'005_GETS_GNS_{layout}_{errp}_{errdist}_p_{pvalue}.xlsx', engine='openpyxl') as writer:

    ws = writer.book.create_sheet(title='Sheet1')

    startrow = 0  # Initial row to start writing the dataframe
    for name, df in results_models.items():
        
        # Write the dataframe name
        ws.cell(row=startrow + 1, column=1).value = f"{name}"
        
        # Write the dataframe content
        df.round(3).to_excel(writer, sheet_name='Sheet1', startrow=startrow + 1, index=False)       
        # Update the startrow for the next dataframe
        endrow = startrow + 2 + len(df)
        startrow += len(df) + 3  
    
    # Apply conditional formatting to columns 3, 4, and 5 for specific value ranges
    # Define fonts for each color
    red_font = Font(color='FF0000')  
    wine_font = Font(color='722F37')   
    blue_font = Font(color='0000FF')  
    green_font = Font(color='00FF00') 
    columns = ['C', 'D', 'E']  # Corresponding to Excel columns 3, 4, and 5
    for col in columns:
        
        # Rule for Red: value > 0.95
        ws.conditional_formatting.add(f'{col}3:{col}{endrow}',
                                      CellIsRule(operator='greaterThan', formula=['0.95'], stopIfTrue=True, font=red_font))
        # Rule for Wine: 0.9 < value <= 0.95
        ws.conditional_formatting.add(f'{col}3:{col}{endrow}',
                                      CellIsRule(operator='between', formula=['0.9', '0.95'], stopIfTrue=True, font=wine_font))
        # Rule for Blue: 0.75 < value <= 0.9
        ws.conditional_formatting.add(f'{col}3:{col}{endrow}',
                                      CellIsRule(operator='between', formula=['0.75', '0.9'], stopIfTrue=True, font=blue_font))
        # Rule for Green: 0.5 < value <= 0.75
        ws.conditional_formatting.add(f'{col}3:{col}{endrow}',
                                      CellIsRule(operator='between', formula=['0.5001', '0.75'], stopIfTrue=True, font=green_font))
